In [ ]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [46]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2024-02-18 03:01:36--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 104.26.2.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M   125MB/s    in 0.2s    

2024-02-18 03:01:37 (125 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
replace BX-Book-Ratings.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [76]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

Calcula o número de avaliações feitas por cada usuário e recebidas por cada livro e armazena em uma série.

In [77]:
usuarios_avalicoes = df_ratings['user'].value_counts()
livros_avaliacoes = df_ratings['isbn'].value_counts()

Filtra as linhas do DataFrame df_ratings onde o número de avaliações feitas por cada usuário é menor que 200.

E faz a mesma coisa com as avaliações recebidas por cada livro porém quando é menor que 100.

In [78]:
df_ratings = df_ratings[~df_ratings['user'].isin(usuarios_avalicoes[usuarios_avalicoes < 200].index)]
df_ratings = df_ratings[~df_ratings['isbn'].isin(livros_avaliacoes[livros_avaliacoes < 100].index)]

Cria uma tabela pivô a partir do DataFrame df_ratings, onde as linhas representam os ISBNs dos livros,
as colunas representam os usuários e os valores representam as classificações (ratings) dadas pelos usuários.
Por fim, preenche valores ausentes com 0.

In [79]:
df_avaliacoes = df_ratings.pivot_table(index='isbn', columns='user', values='rating').fillna(0)

Junta o DataFrame df_table com o DataFrame df_books usando os ISBNs como chave de junção, e substitui os índices da tabela pivô pelos títulos dos livros correspondentes.

In [80]:
df_avaliacoes.index = df_avaliacoes.join(df_books.set_index('isbn'))['title']


  ## Função para fazer as recomendações

  Retorna uma lista de recomendações para um livro específico, ou para um livro semelhante com base em características avaliativas, usando um algoritmo de vizinhos mais próximos com métrica de similaridade cosseno.

    Args:
        book (str): O título do livro para o qual se deseja obter recomendações. Se nenhum livro for fornecido, a função retornará recomendações com base em um livro semelhante encontrado usando o algoritmo.

    Returns:
        list: Uma lista contendo o título do livro especificado e uma lista de até 5 livros recomendados, com suas distâncias relativas ao livro de referência.

In [81]:
def get_recommends(book = ""):
  livros_recomendados = []

  livros_parecidos = NearestNeighbors(n_neighbors=6, metric="cosine").fit(df_avaliacoes.values)

  distances, indices = livros_parecidos.kneighbors([df_avaliacoes.loc[book].values], n_neighbors=6)

  for i in range(1,6):
    livros_recomendados.append([df_avaliacoes.index[indices[0][-i]], distances[0][-i]])

  return [book, livros_recomendados]

In [82]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [["I'll Be Seeing You", 0.8016211], ['The Weight of Water', 0.77085835], ['The Surgeon', 0.7699411], ['I Know This Much Is True', 0.7677075], ['The Lovely Bones: A Novel', 0.7234864]]]
You passed the challenge! 🎉🎉🎉🎉🎉
